In [6]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import scipy.stats
import re
import sys 
import statsmodels.stats.multitest

import cptac
import cptac.utils as u
import plot_utils as p 

/Users/Lindsey/anaconda3/lib/python3.7/site-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [86]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math as math
import scipy.stats
import re
import sys
import statsmodels.stats.multitest
from bokeh.palettes import RdBu
from bokeh.models import LinearColorMapper, ColumnDataSource, ColorBar
from bokeh.models.ranges import FactorRange
from bokeh.plotting import figure, show
from bokeh.io import output_notebook
from bokeh.io import export_png
from bokeh.io import export_svgs
def plotCircleHeatMap ( df, circle_var, color_var, x_axis, y_axis,plot_width= 1000, plot_height = 650, x_axis_lab = "no_label", y_axis_lab = "no_label", show_plot = True, save_png = "plot.png"):
  
    # circle_var designed for pvalues. Normalized by taking log 10 of values and multiplying by 5 
    #added a new column to make the plot size
    
    #df["size"] = (np.-1*log10(df[circle_var]))
    df["size"] = df[circle_var].apply(lambda x: -1*(np.log(x)))
    df["size"] = np.abs(df["size"])
    df['size'] = np.where(df["size"]<0, np.abs(df["size"]), (df["size"])*1.5)
    
    maxval = df[color_var].max()
    minval = df[color_var].min()
    colors = list((RdBu[9]))
    exp_cmap = LinearColorMapper(palette=colors, low = minval, high = maxval)
    p = figure(x_range = FactorRange(), y_range = FactorRange(), plot_width= plot_width, 
               plot_height=plot_height, 
               toolbar_location=None, tools="hover")

    p.scatter(x_axis,y_axis,source=df, fill_alpha=1,  line_width=0, size="size", 
              fill_color={"field":color_var, "transform":exp_cmap})

    p.x_range.factors = sorted(df[x_axis].unique().tolist())
    p.y_range.factors = sorted(df[y_axis].unique().tolist(), reverse = True)
    p.xaxis.major_label_orientation = math.pi/2
    
    if (x_axis_lab != "no_label" ):
        p.xaxis.axis_label = x_axis_lab
    if (x_axis_lab != "no_label" ):   
        p.yaxis.axis_label = y_axis_lab

    bar = ColorBar(color_mapper=exp_cmap, location=(0,0))
    p.add_layout(bar, "right")
    #return df
    if show_plot:
        output_notebook()
        show(p)
      
    if save_png != "plot.png":
        export_png(p, filename= save_png)

In [54]:
coag_genes = ['CPB2','ITGAM','CFH','C1S','C1R','PROS1','SERPINE1','ITGB2','F13A1','C4BPA','PLG','C4BPB','C8B','C8A','C2','C3','VTN','C5','C4A',"C4B",'C8G','C6','C7','C9',"CFG","CD59",'CD55','CD46','PLAU','VSIG4','A2M','MBL2','FGB','FGA','SERPINB2','F10','SERPIND1','FGG','F11','PLAUR','F2','F5','PROCR','F9','SERPING1','MASP2','F13B','MASP1','KLKB1']
coag_column_names = []
for gene in coag_genes:
    gene += "_proteomics"
    coag_column_names.append(gene)



In [55]:
df_FDR_append = pd.read_csv("../Step3.2_combining_pearson_dfs/csv_files/pancan_EGFR_pearson_sig_all_prot_append_FDR.csv")
df_FDR_append = df_FDR_append.drop(['Unnamed: 0'], axis=1)

df_FDR_coag = df_FDR_append[df_FDR_append.Comparison.isin(coag_column_names)]
df_FDR_coag

,Comparison,Correlation,P_value,Cancer Type
77,PROCR_proteomics,-0.470784,8.763500e-07,GBM
196,PLAUR_proteomics,-0.425639,1.122876e-05,GBM
244,SERPINB2_proteomics,-0.482756,1.749352e-05,GBM
305,FGB_proteomics,-0.404936,3.218521e-05,GBM
350,FGG_proteomics,-0.396275,4.899957e-05,GBM
...,...,...,...,...
77622,VTN_proteomics,-0.034665,7.192040e-01,Lscc
77904,ITGB2_proteomics,-0.030324,7.531604e-01,Lscc
78698,MASP2_proteomics,-0.016994,8.601246e-01,Lscc
79271,A2M_proteomics,0.008291,9.314930e-01,Lscc


In [87]:
plotCircleHeatMap(df_FDR_coag, "P_value","Correlation","Comparison","Cancer Type")

/Users/Lindsey/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/Users/Lindsey/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/Users/Lindsey/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: 

Loading BokehJS ...

In [80]:
df_FDR_wide = pd.read_csv("../Step3.2_combining_pearson_dfs/csv_files/pancan_EGFR_all_FDR_wide.csv")
df_FDR_wide = df_FDR_wide.drop(['Unnamed: 0'], axis=1)

df_FDR_wide_coag = df_FDR_wide[df_FDR_wide.Comparison.isin(coag_column_names)]

In [81]:
df_FDR_wide_coag

,Comparison,Correlation_Gbm,P_value_Gbm,Correlation_kidney,P_value_kidney,Correlation_Ovar,P_value_Ovar,Correlation_Brca,P_value_Brca,Correlation_Luad,P_value_Luad,Correlation_hnscc,P_value_hnscc,Correlation_colon,P_value_colon,Correlation_Lscc,P_value_Lscc
77,PROCR_proteomics,-0.470784,8.763500e-07,0.284925,0.002553,NaN,NaN,0.336022,2.412023e-04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
196,PLAUR_proteomics,-0.425639,1.122876e-05,NaN,NaN,NaN,NaN,0.467428,1.389846e-07,NaN,NaN,0.278504,3.363640e-03,NaN,NaN,NaN,NaN
244,SERPINB2_proteomics,-0.482756,1.749352e-05,NaN,NaN,NaN,NaN,0.297636,9.022791e-03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
305,FGB_proteomics,-0.404936,3.218521e-05,NaN,NaN,NaN,NaN,0.303047,9.944820e-04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
350,FGG_proteomics,-0.396275,4.899957e-05,NaN,NaN,NaN,NaN,0.284354,2.070831e-03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
445,FGA_proteomics,-0.380889,1.005188e-04,NaN,NaN,NaN,NaN,0.264073,4.348155e-03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
482,C2_proteomics,-0.373811,1.382574e-04,NaN,NaN,0.368573,0.000607,0.267613,5.781507e-03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
504,F13A1_proteomics,-0.371427,1.536700e-04,NaN,NaN,NaN,NaN,0.235714,1.121352e-02,-0.262430,0.005611,NaN,NaN,NaN,NaN,NaN,NaN
532,F10_proteomics,-0.367388,1.834809e-04,NaN,NaN,NaN,NaN,0.306660,8.581297e-04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
631,C8A_proteomics,-0.354680,3.157269e-04,NaN,NaN,0.430970,0.000048,0.244053,8.577330e-03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
